In [1]:
    import IPython, sys, pyld, types, inspect, pathlib, abc, itertools, urllib, importnb,contextlib, collections, requests_file, requests_cache, fnmatch, traitlets, mimetypes, operator, importlib, requests, requests_cache, urllib, ast, abc,  itertools, mimetypes, io, functools, typing
    if __name__ == '__main__':
        %reload_ext pidgin
    shell = get_ipython()
    CTX, ANNO = '@context __annotations__'.split()
    __all__ = 'ctx', 

In [2]:
## Mime formatting session

In [3]:
    with __import__('importnb').Notebook():
        try: from . import adapters
        except: import adapters

In [4]:
    def expand(str, ctx):
        object = pyld.jsonld.expand({str: "", "@context": ctx})
        if object: return next(iter(object[0]))
        return str

In [5]:
    def treat_path(object):
        try:
            path = pathlib.Path(object)
            if path.exists(): return str(path.absolute())
        except: ...
        return object


In [6]:
    class AnnotationFormatter(IPython.core.formatters.BaseFormatter, requests.Session):
        mimetypes = traitlets.Instance(mimetypes.MimeTypes)
        
        @traitlets.default('mimetypes')
        def _default_mimetypes(Mime): return mimetypes.MimeTypes()
        
        adapter_list = traitlets.List()
        @traitlets.default('adapter_list')
        def _default_adapters(AnnotationFormatter, *object):
            http = adapters.HttpCache('local_cache')
            for adapter in [adapters.PythonAdapter(), adapters.Noop(), adapters.FileAdapter(), http, adapters.ContentsAdapter()]:
                for protocol in adapter.protocol.split(): object += [protocol, adapter],
            return list(object)
                    
        def __init__(AnnotationFormatter, *args, **kwargs):
            IPython.core.formatters.BaseFormatter.__init__(AnnotationFormatter, **kwargs), requests.Session.__init__(AnnotationFormatter)
            for key, value in AnnotationFormatter.adapter_list: AnnotationFormatter.mount(key, value)
            AnnotationFormatter.hooks['response'].append(AnnotationFormatter.post_response)
            
        def post_response(AnnotationFormatter, response, *x, **kwargs):
            response.headers['Content-Type'] =  AnnotationFormatter.mimetypes.guess_type(response.url)[0] or response.headers.get('Content-Type', '').partition(';')[0]
            try:
                response.callable= AnnotationFormatter.wrapped_request_response(AnnotationFormatter.lookup(response.headers['Content-Type']))
            except KeyError: ...
            return response            
        
        def __call__(AnnotationFormatter, object, ctx = None):
            return AnnotationFormatter.get(expand(treat_path(object), ctx or AnnotationFormatter.parent.user_ns.get(ANNO, {}))
                                          ) if isinstance(object, str) else object
        
        def lookup(AnnotationFormatter, object):
            obj_id = id(object)
            if obj_id in AnnotationFormatter.singleton_printers: return AnnotationFormatter.singleton_printers[obj_id]
            if not isinstance(object, str): object = IPython.core.formatters._get_type(object)
            return AnnotationFormatter.lookup_by_type(object)
    
        def lookup_by_type(AnnotationFormatter, type):
            if isinstance(type, str) and (type, None) in AnnotationFormatter.deferred_printers: return AnnotationFormatter.deferred_printers[(type, None)]
            return super().lookup_by_type(type)
            
        def for_type_by_ext(AnnotationFormatter, ext, func=None):
            return AnnotationFormatter.for_type_by_name(*AnnotationFormatter.mimetypes.guess_type(F"foo.{ext}"), func)

        @staticmethod
        def wrapped_request_response(callable):
            @functools.wraps(callable)
            def call(response):
                if len(inspect.signature(callable).parameters) == 1: return callable(response.request)
                return callable(response.request, response)
            return call

In [7]:
    def show_anno_response(response):
        callable = getattr(response, 'callable', None)
        response.object = getattr(response, 'object', None)
        if object is not None: response.object = callable(response) if callable else response        
        if response.status_code == requests.codes.ok:                    
            if isinstance(response.object, type(response)):
                try: return IPython.get_ipython().display_formatter.format(IPython.display.JSON(response.json())) 
                except: ...
            else: return IPython.get_ipython().display_formatter.format(response.object)                 
        else: return IPython.display.Markdown(response.url)

In [8]:
    def get_name(str):
        if '://' in str: return urllib.parse.parse_qs(urllib.parse.urlparse(str).query)['name'][0]
        return str

In [9]:
        def annotate_annotations(shell, object=None):
            if object is None: object = shell.user_module
            if not hasattr(object, ANNO): return
            shell_context = dict(shell.user_ns.get(ANNO, {})).get(CTX, {})
            different = {}
            annotations = dict(getattr(object, ANNO, {}))
            context = {**shell_context, **annotations.pop(CTX, {})}
            for key, value in annotations.items():
                if isinstance(value, str):
                    annotations[key] = treat_path(value)
                if not isinstance(key, str): different[key] = value
                else: context[key] = {"@type": "@vocab", "@id": F"""file://{shell.kernel.parent.connection_file}?name={key}"""}                                    
                    
            for key in different: annotations.pop(key)
            expand = pyld.jsonld.expand({CTX: context, **annotations})
            if expand:
                for key, value in expand[0].items():
                    key, value = get_name(key), value[0]['@id']
                    IPython.display.display(IPython.display.Markdown(F"#### [{key}]({value})"),shell.annotation_formatter(value, ctx=context))
                return
            else: ...

In [10]:
    def load_ipython_extension(shell):
        shell.add_traits(annotation_formatter=traitlets.Instance(AnnotationFormatter, tuple(), {'parent': shell}, allow_none=True))
        shell.display_formatter.mimebundle_formatter.for_type(types.ModuleType, functools.partial(annotate_annotations, shell))   
        shell.display_formatter.mimebundle_formatter.for_type(type, functools.partial(annotate_annotations, shell))           
        shell.display_formatter.mimebundle_formatter.for_type(requests.Response, show_anno_response)   

In [11]:
    def unload_ipython_extension(shell):
        null = lambda x: None
        shell.display_formatter.mimebundle_formatter.for_type(types.ModuleType, null)   
        shell.display_formatter.mimebundle_formatter.for_type(type, null)           
        shell.display_formatter.mimebundle_formatter.for_type(requests.Response, null)   

In [12]:
    if __name__ == '__main__':
        load_ipython_extension(get_ipython())

In [13]:
    class Context(abc.ABCMeta):
        def __new__(cls, name, bases=None, kwargs=None, **context):
            if bases or kwargs:
                annotations = kwargs.pop(ANNO, {})
                return type(name, bases, {**kwargs, '__annotations__': {**annotations, CTX: context}})
            return type(name)
    ctx = {'metaclass': Context}